In [9]:
from ctypes import *
import time
from sys import path
from os import sep
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

dwf = cdll.LoadLibrary("libdwf.so")
constants_path = "/usr/share/digilent/waveforms/samples/py"
path.append(constants_path)
import dwfconstants as constants

In [10]:
dwf.FDwfDeviceCloseAll()
filter_flags = c_int(constants.enumfilterType.value | constants.enumfilterUSB.value)
device_count = c_int()
dwf.FDwfEnum(filter_flags, byref(device_count))

hdwf = c_int()
dwf.FDwfDeviceOpen(c_int(-1), byref(hdwf))
if hdwf.value == 0:
    raise RuntimeError("Failed to open device.")
print(hdwf.value)

dwf.FDwfAnalogIOReset(hdwf)
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(1), c_double(3.0)) #setting VDD = 3V
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(0), c_double(1))
dwf.FDwfAnalogIOEnableSet(hdwf, c_int(1))
time.sleep(0.5)

vpos = c_double()
dwf.FDwfAnalogIOStatus(hdwf)
dwf.FDwfAnalogIOChannelNodeStatus(hdwf, c_int(0), c_int(1), byref(vpos))
print(f"Power: +{vpos.value:.2f}V")

1
Power: +3.01V


In [15]:
DEVICE_ADDR = 0x52
ENABLE_REG = 0x80
ATIME_REG = 0x81
WTIME_REG = 0x83
PERS_REG = 0x8C
CONTROL_REG = 0x8F
CDATAL_REG = 0x94
CDATAH_REG = 0x95
RDATAL_REG = 0x96
RDATAH_REG = 0x97
GDATAL_REG = 0x98
GDATAH_REG = 0x99
BDATAL_REG = 0x9A
BDATAH_REG = 0x9B

def write_register(reg, value):
    iNak = c_int()
    rgTX = (c_ubyte * 2)(reg, value)
    dwf.FDwfDigitalI2cWrite(hdwf, c_int(DEVICE_ADDR), rgTX, c_int(2), byref(iNak))
    return iNak.value == 0

def read_register(reg):
    iNak = c_int()
    rgTX = (c_ubyte * 1)(reg)
    dwf.FDwfDigitalI2cWrite(hdwf, c_int(DEVICE_ADDR), rgTX, c_int(1), byref(iNak))
    if iNak.value == 0:
        rgRX = (c_ubyte * 1)()
        dwf.FDwfDigitalI2cRead(hdwf, c_int(DEVICE_ADDR), rgRX, c_int(1), byref(iNak))
        if iNak.value == 0:
            return rgRX[0]
    return None

dwf.FDwfDigitalI2cRateSet(hdwf, c_double(100e3))
dwf.FDwfDigitalI2cSclSet(hdwf, c_int(0))
dwf.FDwfDigitalI2cSdaSet(hdwf, c_int(1))
iNak = c_int()
dwf.FDwfDigitalI2cClear(hdwf, byref(iNak))

1

In [ ]:
write_register(ENABLE_REG, 0x01) #Power on / PON = 1
time.sleep(0.01)
write_register(ENABLE_REG, 0x13) #Now, also RGBC enable / PON = 1, AEN = 1, AIEN = 1
write_register(PERS_REG, 0x00)
write_register(ATIME_REG, 0xF6) #To run for 24ms, upto a max count of 10,240
write_register(CONTROL_REG, 0x02) #Setting AGAIN=16

True

In [13]:
#Function to test the sensor output
def read_colors():
    time.sleep(0.1)
    
    clear_low = read_register(CDATAL_REG)
    clear_high = read_register(CDATAH_REG)
    red_low = read_register(RDATAL_REG)
    red_high = read_register(RDATAH_REG)
    green_low = read_register(GDATAL_REG)
    green_high = read_register(GDATAH_REG)
    blue_low = read_register(BDATAL_REG)
    blue_high = read_register(BDATAH_REG)
    
    if all(val is not None for val in [clear_low, clear_high, red_low, red_high, green_low, green_high, blue_low, blue_high]):
        clear = (clear_high << 8) | clear_low
        red = (red_high << 8) | red_low
        green = (green_high << 8) | green_low
        blue = (blue_high << 8) | blue_low
        return {'clear': clear, 'red': red, 'green': green, 'blue': blue}
    return None

In [14]:
def cleanup():
    dwf.FDwfDeviceClose(hdwf)

In [8]:
cleanup()
#call it everytime after work is done

In [17]:
c = read_colors()
baseline = c['clear']
print(f"Baseline Light Level: {baseline}")

#Setting thresholds to force interrupt
th_high = max(0, baseline - 50)
th_low = 0

print(f"Setting high threshold value to: {th_high} to force violation")

#Writing low threshold value
write_register(0x84, th_low & 0xFF)
write_register(0x85, (th_low >> 8) & 0xFF)

#Writing high threshold value
write_register(0x86, th_high & 0xFF)
write_register(0x87, (th_high >> 8) & 0xFF)

time.sleep(0.1)

status_val = read_register(0x93)

is_triggered = (status_val & 0x10) != 0

print(f"Status Reg: 0x{status_val:02x}")
print(f"Interrupt Bit (AINT): {'Asserted (Success)' if is_triggered else 'Missing (Fail)'}")

if is_triggered:
    print("Reseting the forced thresholds")
    write_register(0x86, 0xFF)
    write_register(0x87, 0xFF)
    print("Sending 'Clear Interrupt Command")

    iNak = c_int()
    rgTX = (c_ubyte * 1)(0xE6)
    dwf.FDwfDigitalI2cWrite(hdwf, c_int(DEVICE_ADDR), rgTX, c_int(1), byref(iNak))

    #Verifying clear, same as before
    time.sleep(0.05)
    status_val_new = read_register(0x93)
    is_still_triggered = (status_val_new & 0x10) != 0

    print(f"Status Reg after Clear: 0x{status_val_new:02x}")
    if not is_still_triggered:
        print("Result: PASS (Interrupt Triggered and CLeared)")
    else:
        print("Result: FAIL (Interrupt did not clear)")

else:
    print("Result: FAIL (Interrupt did not trigger)")
    

Baseline Light Level: 552
Setting high threshold value to: 502 to force violation
Status Reg: 0x11
Interrupt Bit (AINT): Asserted (Success)
Reseting the forced thresholds
Sending 'Clear Interrupt Command
Status Reg after Clear: 0x11
Result: FAIL (Interrupt did not clear)
